In [1]:
!date

Wed Oct  3 18:15:36 CST 2018


In [1]:
import os
import os.path as osp

import requests as req
import pandas as pd

/Users/semio/.pyenv/versions/3.6.2/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
from urllib.parse import urljoin

In [3]:
from pprint import pprint

In [4]:
from io import BytesIO, StringIO

In [5]:
import numpy as np

In [6]:
np.random.seed(999)

In [7]:
def api_path(api_base, *args):
    res = api_base
    for arg in args:
        if arg.startswith('/'):
            res = osp.join(res, arg[1:])
        else:
            res = osp.join(res, arg)
    return res

In [8]:
api_base =  "https://unstats.un.org"

In [9]:
api_path(api_base, "/SDGAPI", "/v1/sdg/Series/List")

'https://unstats.un.org/SDGAPI/v1/sdg/Series/List'

In [10]:
j = req.get(api_path(api_base, "/SDGAPI", "/v1/sdg/Series/List"), params={'allreleases': False}).json()

In [13]:
len(j)

355

In [14]:
_is = np.random.randint(0, len(j), size=10)

In [15]:
for i in _is:
    pprint(j[i])
    print()

{'code': 'EN_WBE_NDWTL',
 'description': 'Nationally derived extent of wetlands (square kilometres)',
 'goal': ['6'],
 'indicator': ['6.6.1'],
 'release': '2018.Q4.G.01',
 'target': ['6.6'],
 'uri': '/v1/sdg/Series/EN_WBE_NDWTL'}

{'code': 'SI_COV_DISAB',
 'description': 'Proportion of population with severe disabilities receiving '
                'disability cash benefit (%)',
 'goal': ['1'],
 'indicator': ['1.3.1'],
 'release': '2018.Q4.G.01',
 'target': ['1.3'],
 'uri': '/v1/sdg/Series/SI_COV_DISAB'}

{'code': 'ER_PTD_MOTN',
 'description': 'Average proportion of Mountain Key Biodiversity Areas (KBAs) '
                'covered by protected areas (%)',
 'goal': ['15'],
 'indicator': ['15.4.1'],
 'release': '2018.Q4.G.01',
 'target': ['15.4'],
 'uri': '/v1/sdg/Series/ER_PTD_MOTN'}

{'code': 'VC_VAW_PHYPYV',
 'description': 'Proportion of children aged 1-14 years who experienced '
                'physical punishment and/or psychological aggression by '
                'caregivers in

In [16]:
j[:10]

[{'goal': ['15', '15'],
  'target': ['15.a', '15.b'],
  'indicator': ['15.a.1', '15.b.1'],
  'release': '2018.Q4.G.01',
  'code': 'DC_ODA_BDVDL',
  'description': 'Total official development assistance for biodiversity, by donor countries (millions of constant 2016 United States dollars)',
  'uri': '/v1/sdg/Series/DC_ODA_BDVDL'},
 {'goal': ['3'],
  'target': ['3.b'],
  'indicator': ['3.b.2'],
  'release': '2018.Q4.G.01',
  'code': 'DC_TOF_HLTHNT',
  'description': 'Total official development assistance to medical research and basic heath sectors, net disbursement, by recipient countries (millions of constant 2016 United States dollars)',
  'uri': '/v1/sdg/Series/DC_TOF_HLTHNT'},
 {'goal': ['9'],
  'target': ['9.a'],
  'indicator': ['9.a.1'],
  'release': '2018.Q4.G.01',
  'code': 'DC_TOF_INFRAL',
  'description': 'Total official flows for infrastructure, by recipient countries (millions of constant 2016 United States dollars)',
  'uri': '/v1/sdg/Series/DC_TOF_INFRAL'},
 {'goal': ['4'],

In [11]:
csv_api = api_path(api_base, "/SDGAPI", "/v1/sdg/Series/DataCSV")

In [16]:
csv = req.post(csv_api, data={'seriesCodes': "EN_MAT_DOMCMPC"})

In [17]:
with open('../source/EN_MAT_DOMCMPC.csv', 'wb') as f:
    for chunk in csv.iter_content():
        f.write(chunk)
    f.close()

In [26]:
df = pd.read_csv('../source/DC_ODA_BDVDL.csv')

In [27]:
df.duplicated(subset=['GeoAreaCode', 'TimePeriod']).any()  #hmm..

True

In [19]:
from tqdm import tqdm

In [22]:
for v in tqdm(j):
    code = v['code']
    
    f = f'../source/{code}.csv'
    try:
        pd.read_csv(f)
    except KeyboardInterrupt:
        raise
    except:
        csv = req.post(csv_api, data={'seriesCodes': code})

        with open(f'../source/{code}.csv', 'wb') as f:
            for chunk in csv.iter_content():
                f.write(chunk)
            f.close()

100%|██████████| 355/355 [04:06<00:00,  1.44it/s] 
